In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pymysql

In [3]:
import pymysql
from datetime import datetime

mydb = pymysql.connect(
    host="remotemysql.com",
    user="jsdmPgLJ0k",
    password="t0XUXdJsm9",
    database="jsdmPgLJ0k",
)
mycursor = mydb.cursor()



In [4]:
!pip install mtcnn

In [5]:
!pip install expiringdict

In [6]:
from random import choice
from numpy import load
from numpy import expand_dims
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from matplotlib import pyplot
# load faces
data = load('/content/drive/My Drive/Metro project/data.npz')
testX_faces = data['arr_2']
# load face embeddings
data = load('/content/drive/My Drive/Metro project/Data-faces-embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
# normalize input vectors
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)
# fit model
model1 = SVC(kernel='linear', probability=True)
model1.fit(trainX, trainy)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from mtcnn.mtcnn import MTCNN
import cv2
from numpy import load
from numpy import expand_dims
from numpy import savez_compressed
from keras.models import load_model
from expiringdict import ExpiringDict
curr_pass= ExpiringDict(max_len=100, max_age_seconds=600)
detector=MTCNN()

def get_embedding(model, face_pixels):
    face_pixels = face_pixels.astype('float32')
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    samples = expand_dims(face_pixels, axis=0)
    yhat = model.predict(samples)
    return yhat[0]


cap = cv2.VideoCapture("/content/drive/MyDrive/Metro project/test2.mp4")
while True: 
    #Capture frame-by-frame
    ret, frame = cap.read()
    #Use MTCNN to detect faces
    result = detector.detect_faces(frame)
    
    if result != []:
        for person in result:
            bounding_box = person['box']
            keypoints = person['keypoints']
            for i in range(0,3):
                if(bounding_box[i]<0):
                    bounding_box[i]=0
            # print("Printing BB")
            # print(bounding_box)
            cv2.rectangle(frame,
                          (bounding_box[0], bounding_box[1]),
                          (bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),
                          (0,155,255),
                          2)
            x,y,w,h = bounding_box
            cropped_array = frame[y:y+h, x:x+w]
            numarr=asarray(cropped_array)
            model = load_model('/content/drive/My Drive/Metro project/keras-facenet/model/facenet_keras.h5')
            embed=get_embedding(model,numarr)
            samples = expand_dims(embed, axis=0)
            yhat_class = model1.predict(samples)
            yhat_prob = model1.predict_proba(samples)


# get name
            class_index = yhat_class[0]
            class_probability = yhat_prob[0,class_index] * 100
            predict_names = out_encoder.inverse_transform(yhat_class)
            if predict_names[0] not in curr_pass.keys() :
              curr_pass[predict_names[0]]=1
              if predict_names[0]=='Nitya':
                PID=31
              elif predict_names[0]=='Bhumi':
                PID=32
              elif predict_names[0]=='Khush':
                PID=33
              elif predict_names[0]=='Likith':
                PID=34
              StartID = 1  # StartStationID
              mycursor.execute("""INSERT INTO TravelRecord (PID,StartID) VALUES ('%s', '%s')""" % (
                  PID, StartID))
              mydb.commit()
              print(mycursor.rowcount, "record inserted.")
            #   print('Predicted: %s (%.3f)' % (predict_names[0], class_probability))
            # print(curr_pass)
            # print(model1.predict(samples))
            # cv2.imshow("cropped", cropped_array)
            # print(numarr)
    if cv2.waitKey(1) &0xFF == ord('q'):
        break
#When everything's done, release capture

cap.release()
cv2.destroyAllWindows()

1 record inserted.
